In [19]:
#初始化程序
import tecplot as tp
from tecplot.constant import PlotType,SliceSurface,ContLegendLabelLocation
tp.session.connect()
frame = tp.active_frame()
dataset = frame.dataset

Connecting to Tecplot 360 TecUtil Server on:
    tcp://localhost:7600
Connection established.


In [28]:
#坐标转换
def axes_transform(frame = tp.active_frame(),transform = {'z':'y','y':'z'},init = True):
    axes = frame.plot().axes #笛卡尔三维坐标系
    plot = frame.plot()
    for i in transform:
        if i == 'x' or i == 'X':
            axes.x_axis.variable = dataset.variable(f'{transform[i]}')
        elif i == 'y' or i == 'Y':
            axes.y_axis.variable = dataset.variable(f'{transform[i]}')
        elif i == 'z' or i == 'Z':
            axes.z_axis.variable = dataset.variable(f'{transform[i]}')
    if init: #初始化坐标轴，重设缩放
        axes.reset_range()
        axes.reset_scale()
        plot.view.fit()

#求解速度场
def velocity_field():
    '''
    数据转换函数，使用方式参考：
    二氧化碳浓度场：
    tp.data.operate.execute_equation('{CO2 (ppm)} = {Mass fraction of co2}*1E6')
    '''
    tp.data.operate.execute_equation('{Velocity (m/s)} = SQRT({X Velocity}**2 + {Y Velocity}**2 + {Z Velocity}**2)')

#创建剖切面云图
def creat_slice(frame = tp.active_frame(),variable = 'Static Pressure',
                translucency = True,shade = False,slices = True):
    
    plot = frame.plot()
    plot.use_translucency = translucency
    plot.contour(0).variable = dataset.variable(variable)
    plot.show_shade = shade
    plot.show_slices = slices
    plot.slice(0).orientation = SliceSurface.XPlanes
    plot.slice(1).orientation = SliceSurface.YPlanes
    plot.slice(2).orientation = SliceSurface.ZPlanes
    slices = plot.slices(0,1,2)
    slices.show = True
    slices.edge.show = True
    slices.edge.line_thickness = 0.25
    slices.effects.use_translucency = True
    return slices

#重置剖切面位置
def reset_slices(plot = tp.active_frame().plot()):
    axes = plot.axes
    l_axes = [axes.x_axis,axes.y_axis,axes.z_axis]
    slices = creat_slice()
    temp = ()
    j = 0
    for each in axes:
        temp = temp + ((each.max + each.min)/2,)
        init_origin = [0,0,0]
        init_origin[j] = temp[j]
        slices[j].origin = tuple(init_origin)
        j += 1

#云图图例格式化
def legend_init(frame = tp.active_frame(),fonts = 'Time New Roman'):
    legend = frame.plot().contour(0).legend
    legend.show = True
    legend.vertical = False
    legend.label_location = ContLegendLabelLocation.ContourLevels
    legend.position = (100,100)
    legend.box.box_type = tp.constant.TextBox.None_
    legend.number_font.typeface = fonts
    legend.number_font.size = 2.5
    legend.header.font.typeface = fonts
    legend.header.font.size = 2.5

#方向标格式化
def axes_init(frame = tp.active_frame()):
    plot = frame.plot()
    plot.axes.orientation_axis.show = True
    plot.axes.orientation_axis.position = 7,7
    plot.axes.orientation_axis.size = 7

In [31]:
a = frame.plot().axes.x_axis.max
b = frame.plot().axes.x_axis.min
test = a+b
test/2

-0.5034360885620122

In [24]:
plot.show_slices = True

In [29]:
axes_transform()
velocity_field()
creat_slice()
legend_init()
axes_init()